In [1]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime

# CẤU HÌNH - THAY ĐỔI TẠI ĐÂY
PROJECT_ID = 'ee-bonglantrungmuoi'  # Thay bằng GEE project ID của bạn
ASSET_PATH = 'projects/ee-bonglantrungmuoi/assets/ca_mau'  # Đường dẫn asset shapefile
OUTPUT_FOLDER = './sentinel-1'  # Thư mục lưu dữ liệu

# Tham số thời gian
START_DATE = '2023-12-01'
END_DATE = '2024-01-31'

# Tham số Sentinel-1
ORBIT_DIRECTION = 'DESCENDING'  # ASCENDING hoặc DESCENDING
INSTRUMENT_MODE = 'IW'
RESOLUTION = 10  # Độ phân giải (m)

# Khởi tạo Google Earth Engine
ee.Initialize(project=PROJECT_ID)

# Tạo thư mục lưu dữ liệu
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print(f"=== CẤU HÌNH TẢI SENTINEL-1 ===")
print(f"- Project: {PROJECT_ID}")
print(f"- Asset: {ASSET_PATH}")
print(f"- Thời gian: {START_DATE} → {END_DATE}")
print(f"- Orbit: {ORBIT_DIRECTION}")
print(f"- Độ phân giải: {RESOLUTION}m")
print(f"- Thư mục lưu: {OUTPUT_FOLDER}")
print("✅ Sẵn sàng tải dữ liệu!")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


=== CẤU HÌNH TẢI SENTINEL-1 ===
- Project: ee-bonglantrungmuoi
- Asset: projects/ee-bonglantrungmuoi/assets/ca_mau
- Thời gian: 2023-12-01 → 2024-01-31
- Orbit: DESCENDING
- Độ phân giải: 10m
- Thư mục lưu: ./sentinel-1
✅ Sẵn sàng tải dữ liệu!


In [2]:
# Tải shapefile và tạo collection
print("=== TẢI SHAPEFILE VÀ TẠO COLLECTION ===")

# Tải shapefile từ GEE Asset
study_area = ee.FeatureCollection(ASSET_PATH)
geometry = study_area.geometry()

# Tự động lấy tên từ asset path
asset_name = ASSET_PATH.split('/')[-1]  # Lấy phần cuối của path
REGION_NAME = asset_name  # Cập nhật tên vùng nghiên cứu

print(f"✅ Đã tải shapefile: {ASSET_PATH}")
print(f"🏷️ Tên vùng nghiên cứu: {REGION_NAME}")

# Tạo Sentinel-1 collection
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filterBounds(geometry)
              .filterDate(START_DATE, END_DATE)
              .filter(ee.Filter.eq('orbitProperties_pass', ORBIT_DIRECTION))
              .filter(ee.Filter.eq('instrumentMode', INSTRUMENT_MODE))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

# Thông tin collection
total_images = collection.size().getInfo()
print(f"✅ Tổng số ảnh tìm thấy: {total_images}")

if total_images == 0:
    print("❌ Không tìm thấy ảnh nào! Kiểm tra lại thời gian và vùng nghiên cứu.")
else:
    print("🎯 Sẵn sàng tải dữ liệu!")

=== TẢI SHAPEFILE VÀ TẠO COLLECTION ===
✅ Đã tải shapefile: projects/ee-bonglantrungmuoi/assets/ca_mau
🏷️ Tên vùng nghiên cứu: ca_mau
✅ Tổng số ảnh tìm thấy: 10
🎯 Sẵn sàng tải dữ liệu!
✅ Tổng số ảnh tìm thấy: 10
🎯 Sẵn sàng tải dữ liệu!


In [3]:
# Function tải dữ liệu
print("=== THIẾT LẬP FUNCTION TẢI DỮ LIỆU ===")

def download_monthly_composite(year, month):
    """
    Tải composite tháng
    
    Args:
        year: Năm
        month: Tháng
    """
    # Tạo khoảng thời gian tháng
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    # Lọc ảnh theo tháng
    monthly_collection = collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu không
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
        return None
    
    print(f"📊 Tháng {year}-{month:02d}: {count} ảnh")
    
    # Tạo composite
    composite = (monthly_collection
                .select(['VV', 'VH'])
                .median()
                .clip(geometry))

    # Tạo tên file
    filename = f'S1_{REGION_NAME}_{year}_{month:02d}_{RESOLUTION}m.tif'
    filepath = os.path.join(OUTPUT_FOLDER, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        # Tải file
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=RESOLUTION,
            region=geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra kết quả
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Thành công: {file_size:.1f} MB")
            return filepath
        else:
            print(f"❌ Thất bại: Không tạo được file")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi: {str(e)}")
        return None

def download_single_image(index):
    """
    Tải ảnh riêng lẻ theo index
    
    Args:
        index: Chỉ số ảnh trong collection (bắt đầu từ 0)
    """
    try:
        # Lấy ảnh theo index
        image = ee.Image(collection.toList(1, index).get(0))
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        
        # Xử lý ảnh
        processed = (image
                    .select(['VV', 'VH'])
                    .clip(geometry))
        
        # Tên file
        filename = f'S1_{REGION_NAME}_{date}_{RESOLUTION}m.tif'
        filepath = os.path.join(OUTPUT_FOLDER, filename)
        
        print(f"🔄 Đang tải {filename}...")
        
        # Tải file
        geemap.download_ee_image(
            image=processed,
            filename=filepath,
            scale=RESOLUTION,
            region=geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra kết quả
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Thành công: {file_size:.1f} MB")
            return filepath
        else:
            print(f"❌ Thất bại: Không tạo được file")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi: {str(e)}")
        return None

print("✅ Functions sẵn sàng sử dụng!")
print("\n💡 Cách sử dụng:")
print("   download_monthly_composite(2023, 12)  # Tải composite tháng 12/2023")
print("   download_single_image(0)             # Tải ảnh đầu tiên trong collection")

=== THIẾT LẬP FUNCTION TẢI DỮ LIỆU ===
✅ Functions sẵn sàng sử dụng!

💡 Cách sử dụng:
   download_monthly_composite(2023, 12)  # Tải composite tháng 12/2023
   download_single_image(0)             # Tải ảnh đầu tiên trong collection


In [4]:
# Cài đặt thư viện cần thiết (chỉ chạy một lần)
print("=== KIỂM TRA VÀ CÀI ĐẶT THƯ VIỆN ===")

try:
    import geedim
    print("✅ geedim đã được cài đặt")
except ImportError:
    print("🔧 Đang cài đặt geedim...")
    import subprocess
    import sys
    
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "geedim"])
        print("✅ Đã cài đặt geedim thành công")
        import geedim  # Import sau khi cài đặt
    except Exception as e:
        print(f"❌ Lỗi cài đặt geedim: {str(e)}")
        print("💡 Thử chạy lệnh: pip install geedim")

print("🎯 Sẵn sàng tải dữ liệu!")

=== KIỂM TRA VÀ CÀI ĐẶT THƯ VIỆN ===
✅ geedim đã được cài đặt
🎯 Sẵn sàng tải dữ liệu!
✅ geedim đã được cài đặt
🎯 Sẵn sàng tải dữ liệu!


In [5]:
# TẢI DỮ LIỆU
print("=== TẢI DỮ LIỆU SENTINEL-1 ===")

# Chuyển đổi thời gian từ string sang datetime
from datetime import datetime
start_dt = datetime.strptime(START_DATE, '%Y-%m-%d')
end_dt = datetime.strptime(END_DATE, '%Y-%m-%d')

# CẤU HÌNH TẢI - THAY ĐỔI TẠI ĐÂY
DOWNLOAD_TYPE = 'individual'  # 'monthly' hoặc 'individual'

print(f"Loại tải: {DOWNLOAD_TYPE}")
print(f"Khoảng thời gian: {START_DATE} → {END_DATE}")
print(f"Độ phân giải: {RESOLUTION}m")

# Danh sách file đã tải
downloaded_files = []
failed_downloads = []

if DOWNLOAD_TYPE == 'monthly':
    # TẢI COMPOSITE THÁNG theo thời gian đã cấu hình
    months_to_download = []
    
    # Tạo danh sách các tháng cần tải
    current = start_dt.replace(day=1)  # Đầu tháng
    while current <= end_dt:
        months_to_download.append((current.year, current.month))
        # Chuyển sang tháng tiếp theo
        if current.month == 12:
            current = current.replace(year=current.year + 1, month=1)
        else:
            current = current.replace(month=current.month + 1)
    
    total_months = len(months_to_download)
    print(f"\n🚀 Bắt đầu tải {total_months} composite tháng...")
    
    for count, (year, month) in enumerate(months_to_download, 1):
        print(f"\n[{count}/{total_months}] Tháng {year}-{month:02d}")
        
        filepath = download_monthly_composite(year, month)
        
        if filepath:
            downloaded_files.append(filepath)
        else:
            failed_downloads.append(f"{year}-{month:02d}")
        
        # Hiển thị tiến trình
        progress = (count / total_months) * 100
        print(f"Tiến trình: {progress:.1f}%")

else:
    # TẢI ẢNH RIÊNG LẺ
    print(f"\n🚀 Bắt đầu tải {total_images} ảnh riêng lẻ...")
    
    for i in range(total_images):
        print(f"\n[{i+1}/{total_images}] Ảnh #{i}")
        
        filepath = download_single_image(i)
        
        if filepath:
            downloaded_files.append(filepath)
        else:
            failed_downloads.append(f"image_{i}")
        
        # Hiển thị tiến trình
        progress = ((i + 1) / total_images) * 100
        print(f"Tiến trình: {progress:.1f}%")

# TỔNG KẾT
print(f"\n{'='*50}")
print(f"🎉 HOÀN THÀNH!")
print(f"{'='*50}")

if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    
    print(f"✅ Thành công: {len(downloaded_files)} file")
    print(f"📊 Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.1f} GB)")
    print(f"📁 Thư mục: {OUTPUT_FOLDER}")

if failed_downloads:
    print(f"\n❌ Thất bại: {len(failed_downloads)} file")
    if len(failed_downloads) <= 10:
        print("Danh sách thất bại:", failed_downloads)
    else:
        print("Một số thất bại:", failed_downloads[:10], "...")

print(f"\n✅ Kiểm tra kết quả ở cell tiếp theo!")

=== TẢI DỮ LIỆU SENTINEL-1 ===
Loại tải: individual
Khoảng thời gian: 2023-12-01 → 2024-01-31
Độ phân giải: 10m

🚀 Bắt đầu tải 10 ảnh riêng lẻ...

[1/10] Ảnh #0
🔄 Đang tải S1_ca_mau_2023-12-06_10m.tif...
🔄 Đang tải S1_ca_mau_2023-12-06_10m.tif...


Consider adjusting `region`, `scale` and/or `dtype` to reduce the S1_ca_mau_2023-12-06_10m.tif download size (raw: 1.46 GB).


S1_ca_mau_2023-12-06_10m.tif: |          | 0.00/1.46G (raw) [  0.0%] in 00:00 (eta:     ?)

✅ Thành công: 188.1 MB
Tiến trình: 10.0%

[2/10] Ảnh #1
🔄 Đang tải S1_ca_mau_2023-12-06_10m.tif...
🔄 Đang tải S1_ca_mau_2023-12-06_10m.tif...


Consider adjusting `region`, `scale` and/or `dtype` to reduce the S1_ca_mau_2023-12-06_10m.tif download size (raw: 1.46 GB).


...

S1_ca_mau_2024-01-23_10m.tif: |          | 0.00/1.46G (raw) [  0.0%] in 00:00 (eta:     ?)

✅ Thành công: 185.9 MB
Tiến trình: 90.0%

[10/10] Ảnh #9
🔄 Đang tải S1_ca_mau_2024-01-23_10m.tif...
🔄 Đang tải S1_ca_mau_2024-01-23_10m.tif...


Consider adjusting `region`, `scale` and/or `dtype` to reduce the S1_ca_mau_2024-01-23_10m.tif download size (raw: 1.46 GB).


S1_ca_mau_2024-01-23_10m.tif: |          | 0.00/1.46G (raw) [  0.0%] in 00:00 (eta:     ?)

✅ Thành công: 797.7 MB
Tiến trình: 100.0%

🎉 HOÀN THÀNH!
✅ Thành công: 10 file
📊 Tổng dung lượng: 7886.7 MB (7.7 GB)
📁 Thư mục: ./sentinel-1

✅ Kiểm tra kết quả ở cell tiếp theo!


In [6]:
# KIỂM TRA KẾT QUẢ
print("=== KIỂM TRA KẾT QUẢ ===")

def check_files():
    """Kiểm tra file đã tải"""
    if not os.path.exists(OUTPUT_FOLDER):
        print("❌ Thư mục chưa tồn tại")
        return []
    
    files = [f for f in os.listdir(OUTPUT_FOLDER) if f.endswith('.tif')]
    
    if not files:
        print("📂 Chưa có file nào")
        return []
    
    files.sort()  # Sắp xếp theo tên
    
    print(f"📊 Tìm thấy {len(files)} file:")
    
    total_size = 0
    for i, file in enumerate(files, 1):
        filepath = os.path.join(OUTPUT_FOLDER, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        total_size += size
        print(f"   {i:2d}. {file} ({size:.1f} MB)")
    
    print(f"\n📈 Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    return files

def delete_all_files():
    """Xóa tất cả file đã tải"""
    files = [f for f in os.listdir(OUTPUT_FOLDER) if f.endswith('.tif')] if os.path.exists(OUTPUT_FOLDER) else []
    
    if not files:
        print("📂 Không có file để xóa")
        return
    
    confirm = input(f"⚠️  Xóa {len(files)} file? (y/N): ")
    if confirm.lower() == 'y':
        for file in files:
            os.remove(os.path.join(OUTPUT_FOLDER, file))
        print(f"🗑️  Đã xóa {len(files)} file")
    else:
        print("❌ Hủy bỏ")

# Kiểm tra kết quả hiện tại
files = check_files()

if files:
    print(f"\n💡 Sử dụng delete_all_files() để xóa tất cả file")
    
print(f"\n🔧 Thông tin cấu hình:")
print(f"   📁 Thư mục dữ liệu: {OUTPUT_FOLDER}")
try:
    print(f"   🏷️ Tên vùng nghiên cứu: {REGION_NAME}")
except NameError:
    print(f"   🏷️ Tên vùng nghiên cứu: (chưa được thiết lập - chạy cell 2)")
print(f"   📏 Độ phân giải: {RESOLUTION}m")

=== KIỂM TRA KẾT QUẢ ===
📊 Tìm thấy 5 file:
    1. S1_ca_mau_2023-12-06_10m.tif (783.7 MB)
    2. S1_ca_mau_2023-12-18_10m.tif (790.4 MB)
    3. S1_ca_mau_2023-12-30_10m.tif (785.7 MB)
    4. S1_ca_mau_2024-01-11_10m.tif (785.9 MB)
    5. S1_ca_mau_2024-01-23_10m.tif (797.7 MB)

📈 Tổng dung lượng: 3943.4 MB (3.85 GB)

💡 Sử dụng delete_all_files() để xóa tất cả file

🔧 Thông tin cấu hình:
   📁 Thư mục dữ liệu: ./sentinel-1
   🏷️ Tên vùng nghiên cứu: ca_mau
   📏 Độ phân giải: 10m
